# Genie Server API Test

Use this notebook to interact with the FastAPI server exposed in `server.py`. Make sure the server is running (e.g. `uv run uvicorn server:app --reload --port 8000`) before executing the cells below.

## Configuration
Adjust the base URL if the server is running on another host or port.

In [1]:
BASE_URL = "http://127.0.0.1:8001"
BASE_URL


'http://127.0.0.1:8001'

In [4]:
import requests
import time
from pprint import pprint


def create_task(payload: dict) -> dict:
    resp = requests.post(f"{BASE_URL}/tasks", json=payload)
    resp.raise_for_status()
    return resp.json()



def get_task_status(task_id: str) -> dict:
    resp = requests.get(f"{BASE_URL}/tasks/{task_id}")
    if resp.status_code == 404:
        detail = resp.json()
        return {"status": "not_found", "detail": detail}
    resp.raise_for_status()
    return resp.json()



def wait_for_task(task_id: str, interval: float = 2.0) -> dict:
    """Polls the server until the task completes or fails."""
    while True:
        status = get_task_status(task_id)
        pprint(status)
        if status.get("status") in {"completed", "failed", "not_found"}:
            return status
        time.sleep(interval)


## Prepare request payload
Fill in the character/reference information required by your Genie setup.

In [3]:
task_payload = {
    "character_name": "はづき",  # 替换为角色
    "reference_audio_id": "special_communications/490300191/4903001910010",  # 替换为参考音频 ID
    "reference_audio_text": "お誕生日おめでとうございます、\nプロデューサーさん♪",
    "text": "光降りそそぐチャペルで、誓いを胸に輝く花嫁姿。\nハートを射抜くほどに甘い、彼女の永遠の魅力を写し取った一枚が完成しました。",
}


task_payload

{'character_name': 'はづき',
 'reference_audio_id': 'special_communications/490300191/4903001910010',
 'reference_audio_text': 'お誕生日おめでとうございます、\nプロデューサーさん♪',
 'text': '光降りそそぐチャペルで、誓いを胸に輝く花嫁姿。\nハートを射抜くほどに甘い、彼女の永遠の魅力を写し取った一枚が完成しました。'}

## Create a task

In [7]:
create_response = create_task(task_payload)
task_id = create_response["task_id"]
task_id

'はづき_1766497173_0924d41a'

## Poll task status

In [8]:
final_status = wait_for_task(task_id, interval=3)
final_status


{'error': None,
 'pending': 0,
 'save_path': './output/はづき_1766497173_0924d41a.wav',
 'status': 'running'}
{'error': None,
 'pending': 0,
 'save_path': './output/はづき_1766497173_0924d41a.wav',
 'status': 'running'}
{'error': None,
 'pending': 0,
 'save_path': './output/はづき_1766497173_0924d41a.wav',
 'status': 'running'}
{'error': None,
 'pending': 0,
 'save_path': './output/はづき_1766497173_0924d41a.wav',
 'status': 'completed'}


{'status': 'completed',
 'pending': 0,
 'save_path': './output/はづき_1766497173_0924d41a.wav',
 'error': None}

## Manual status check
Use this cell to query any task ID directly.

In [ ]:
some_task_id = task_id  # 或替换为其他任务 ID
get_task_status(some_task_id)
